# Simple CNN with Custom Data

In [1]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline  
print ("Packages loaded")

Packages loaded


# Load data

In [2]:
# Load them!
cwd = os.getcwd()
loadpath = cwd + "/data/data_gray.npz"
l = np.load(loadpath)

# See what's in here
print (l.files)

# Parse data
trainimg = l['trainimg']
trainlabel = l['trainlabel']
testimg = l['testimg']
testlabel = l['testlabel']
imgsize = l['imgsize']
ntrain = trainimg.shape[0]
nclass = trainlabel.shape[1]
dim    = trainimg.shape[1]
ntest  = testimg.shape[0]
print ("%d train images loaded" % (ntrain))
print ("%d test images loaded" % (ntest))
print ("%d dimensional input" % (dim))
print ("Image size is %s" % (imgsize))
print ("%d classes" % (nclass))

['trainlabel', 'trainimg', 'imgsize', 'testimg', 'testlabel']
52 train images loaded
35 test images loaded
4096 dimensional input
Image size is [64 64]
2 classes


# Define CNN architecture 

In [3]:
n_input  = dim
n_output = nclass
weights  = {
    'wc1': tf.Variable(tf.random_normal([7, 7, 1, 128], stddev=0.1)),
    'wc2': tf.Variable(tf.random_normal([3, 3, 128, 128], stddev=0.1)),
    'wd1': tf.Variable(tf.random_normal(
            [imgsize[0]/4*imgsize[1]/4*128, 128], stddev=0.1)),
    'wd2': tf.Variable(tf.random_normal([128, n_output], stddev=0.1))
}
biases   = {
    'bc1': tf.Variable(tf.random_normal([128], stddev=0.1)),
    'bc2': tf.Variable(tf.random_normal([128], stddev=0.1)),
    'bd1': tf.Variable(tf.random_normal([128], stddev=0.1)),
    'bd2': tf.Variable(tf.random_normal([n_output], stddev=0.1))
}
def conv_basic(_input, _w, _b, _keepratio):
    # Input
    _input_r = tf.reshape(_input, shape=[-1, imgsize[0], imgsize[1], 1])
    # Conv1
    _conv1 = tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(_input_r
        , _w['wc1'], strides=[1, 1, 1, 1], padding='SAME'), _b['bc1']))
    _pool1 = tf.nn.max_pool(_conv1, ksize=[1, 2, 2, 1]
        , strides=[1, 2, 2, 1], padding='SAME')
    # mean, var = tf.nn.moments(_pool1, [0, 1, 2])
    # _pool1 = tf.nn.batch_norm_with_global_normalization(
    #     _pool1, mean, var, 1., 0., 1e-7, 0)
    _pool_dr1 = tf.nn.dropout(_pool1, _keepratio)
    # Conv2
    _conv2 = tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(_pool_dr1
        , _w['wc2'], strides=[1, 1, 1, 1], padding='SAME'), _b['bc2']))
    _pool2 = tf.nn.max_pool(_conv2, ksize=[1, 2, 2, 1]
        , strides=[1, 2, 2, 1], padding='SAME')
    # mean, var = tf.nn.moments(_pool2, [0, 1, 2])
    # _pool2 = tf.nn.batch_norm_with_global_normalization(
    #     _pool2, mean, var, 1., 0., 1e-7, 0)
    _pool_dr2 = tf.nn.dropout(_pool2, _keepratio)
    # Vectorize
    _dense1 = tf.reshape(_pool_dr2
                         , [-1, _w['wd1'].get_shape().as_list()[0]])
    # Fc1
    _fc1 = tf.nn.relu(tf.add(tf.matmul(_dense1, _w['wd1']), _b['bd1']))
    _fc_dr1 = tf.nn.dropout(_fc1, _keepratio)
    # Fc2
    _out = tf.add(tf.matmul(_fc_dr1, _w['wd2']), _b['bd2'])
    # Return everything
    out = {
        'input_r': _input_r, 'conv1': _conv1, 'pool1': _pool1
        , 'pool1_dr1': _pool_dr1, 'conv2': _conv2, 'pool2': _pool2
        , 'pool_dr2': _pool_dr2, 'dense1': _dense1, 'fc1': _fc1
        , 'fc_dr1': _fc_dr1, 'out': _out
    }
    return out
print ("Network Ready to Go!")

Network Ready to Go!


# Define functions

In [4]:
# tf Graph input
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_output])
keepratio = tf.placeholder(tf.float32)

# Functions! 
_pred = conv_basic(x, weights, biases, keepratio)['out']
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(_pred, y))
WEIGHT_DECAY_FACTOR = 0.0001
l2_loss = tf.add_n([tf.nn.l2_loss(v) 
            for v in tf.trainable_variables()])
cost = cost + WEIGHT_DECAY_FACTOR*l2_loss
optm = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)
_corr = tf.equal(tf.argmax(_pred,1), tf.argmax(y,1)) # Count corrects
accr = tf.reduce_mean(tf.cast(_corr, tf.float32)) # Accuracy
init = tf.initialize_all_variables()
print ("Functions ready")

Functions ready


# Optimize

In [5]:
# Parameters
training_epochs = 400
batch_size      = 100
display_step    = 40

# Launch the graph
sess = tf.Session()
sess.run(init)

# Training cycle
for epoch in range(training_epochs):
    avg_cost = 0.
    num_batch = int(ntrain/batch_size)+1
    # Loop over all batches
    for i in range(num_batch): 
        randidx = np.random.randint(ntrain, size=batch_size)
        batch_xs = trainimg[randidx, :]
        batch_ys = trainlabel[randidx, :]                
        # Fit training using batch data
        sess.run(optm, feed_dict={x: batch_xs, y: batch_ys
                                  , keepratio:0.5})
        # Compute average loss
        avg_cost += sess.run(cost, feed_dict={x: batch_xs, y: batch_ys
                                , keepratio:1.})/num_batch

    # Display logs per epoch step
    if epoch % display_step == 0 or epoch == training_epochs-1:
        print ("Epoch: %03d/%03d cost: %.9f" % 
               (epoch, training_epochs, avg_cost))
        train_acc = sess.run(accr, feed_dict={x: batch_xs
                                , y: batch_ys, keepratio:1.})
        print (" Training accuracy: %.3f" % (train_acc))
        test_acc = sess.run(accr, feed_dict={x: testimg
                                , y: testlabel, keepratio:1.})
        print (" Test accuracy: %.3f" % (test_acc))

print ("Optimization Finished!")

Epoch: 000/400 cost: 22.932985306
 Training accuracy: 0.460
 Test accuracy: 0.400
Epoch: 040/400 cost: 2.813483953
 Training accuracy: 0.570
 Test accuracy: 0.400
Epoch: 080/400 cost: 2.744574785
 Training accuracy: 0.560
 Test accuracy: 0.429
Epoch: 120/400 cost: 2.657322884
 Training accuracy: 0.840
 Test accuracy: 0.629
Epoch: 160/400 cost: 2.600071192
 Training accuracy: 0.840
 Test accuracy: 0.743
Epoch: 200/400 cost: 2.448372126
 Training accuracy: 0.850
 Test accuracy: 0.800
Epoch: 240/400 cost: 2.369165897
 Training accuracy: 0.880
 Test accuracy: 0.686
Epoch: 280/400 cost: 2.061110258
 Training accuracy: 1.000
 Test accuracy: 0.800
Epoch: 320/400 cost: 1.966778517
 Training accuracy: 1.000
 Test accuracy: 0.743
Epoch: 360/400 cost: 1.928094387
 Training accuracy: 1.000
 Test accuracy: 0.771
Epoch: 399/400 cost: 1.898978710
 Training accuracy: 1.000
 Test accuracy: 0.800
Optimization Finished!


In [6]:
test_acc = sess.run(accr, feed_dict={x: testimg
                    , y: testlabel, keepratio:1.})
print (" Test accuracy: %.3f" % (test_acc))
#sess.close()
#print ("Session closed.")

 Test accuracy: 0.800
